In [1]:
import pandas as pd

Importing data set

In [2]:
X_test_full = pd.read_csv('data_test.csv')
X_train_full = pd.read_csv('data_train.csv')

Dropping observation with incomplete data

In [3]:
X_test = X_test_full.dropna() 
X_train = X_train_full.dropna()

Seperating data set

In [4]:
y = X_train.Y
X_train.drop(['Y'], axis=1, inplace=True)

Invastigating data

In [5]:
print( "Number of variables: ", len(X_train.columns), "\nNumber of observations: ", len(X_train.index))
X_train.info()

Number of variables:  17737 
Number of observations:  644
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Columns: 17737 entries, ENSG00000000003 to ENSG00000266753
dtypes: float64(17737)
memory usage: 87.1 MB


All data is numerical

Computing coefficient of variation for all columns

In [6]:
coef_var = {}
var = X_train.var()
mean = X_train.mean()
for i in range(0, len(X_train.columns)):
    coef_var[i] = var[i]/mean[i]

Selecing 500 columns with highest value of coefficient of variation

In [7]:
top_500_coef_var=sorted(coef_var, key=coef_var.get, reverse=True)[:500]

Coputing correlation of top 500 columns

In [8]:
top_500_cols = X_train.iloc[:, top_500_coef_var]
corr = top_500_cols.corr()
corr


,ENSG00000129824,ENSG00000184292,ENSG00000163993,ENSG00000122862,ENSG00000171345,ENSG00000204287,ENSG00000118785,ENSG00000230937,ENSG00000133169,ENSG00000167644,...,ENSG00000143333,ENSG00000196735,ENSG00000157214,ENSG00000170476,ENSG00000164294,ENSG00000079112,ENSG00000075643,ENSG00000139289,ENSG00000125850,ENSG00000125726
ENSG00000129824,1.000000,-0.172547,-0.154224,0.095800,-0.186533,0.088862,-0.000511,-0.099702,0.105643,-0.177893,...,0.112090,0.128446,-0.012207,0.075844,-0.071932,-0.077886,-0.081175,-0.054768,-0.164021,0.058945
ENSG00000184292,-0.172547,1.000000,0.559029,-0.219109,0.685896,-0.125801,-0.104377,0.709955,-0.386670,0.654923,...,-0.267545,-0.160299,0.196770,-0.193025,0.153813,0.016765,0.379531,0.172817,0.597723,-0.268973
ENSG00000163993,-0.154224,0.559029,1.000000,-0.134828,0.543591,-0.109907,-0.106628,0.424270,-0.306688,0.506648,...,-0.132693,-0.135970,0.200269,-0.166865,-0.078209,0.402660,0.353688,0.099603,0.512415,-0.305832
ENSG00000122862,0.095800,-0.219109,-0.134828,1.000000,-0.285207,0.311946,0.096311,-0.208802,-0.056677,-0.175856,...,-0.028097,0.274049,-0.078186,0.289081,0.020553,-0.167722,-0.062973,-0.065129,-0.392380,0.406093
ENSG00000171345,-0.186533,0.685896,0.543591,-0.285207,1.000000,-0.265346,-0.003087,0.456647,-0.249832,0.733174,...,-0.258963,-0.229703,0.265831,-0.296715,0.121715,0.250129,0.363087,0.165055,0.648500,-0.311482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000079112,-0.077886,0.016765,0.402660,-0.167722,0.250129,-0.091610,-0.113976,-0.161159,-0.117890,0.239679,...,-0.014074,-0.052064,0.204883,-0.046479,-0.168724,1.000000,0.160737,0.039392,0.241669,-0.123707
ENSG00000075643,-0.081175,0.379531,0.353688,-0.062973,0.363087,-0.176922,0.145971,0.230607,-0.356555,0.473375,...,-0.355307,-0.226620,0.270314,-0.323932,0.376451,0.160737,1.000000,0.411549,0.202509,-0.114075
ENSG00000139289,-0.054768,0.172817,0.099603,-0.065129,0.165055,-0.148803,0.215056,0.131172,-0.185868,0.342359,...,-0.413273,-0.209523,0.309539,-0.457291,0.530755,0.039392,0.411549,1.000000,-0.037749,-0.087807
ENSG00000125850,-0.164021,0.597723,0.512415,-0.392380,0.648500,-0.208162,-0.135227,0.426460,-0.109122,0.441673,...,-0.040930,-0.178070,0.112117,-0.182985,-0.131150,0.241669,0.202509,-0.037749,1.000000,-0.398243


Some of the columns are corralated. Due to that fact we have to use regression method that can deal with correlated variables. Lasso and ridge regression are the methods that could help. To know which one of them will perform better we will use Elastic Net model. Elastic net is the compromise between Lasso and ridge regression. Contribution of these two models can be balanced using "l1_ratio". I will also use cross validation to obtain better results.

In [9]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def get_score(l1_ratio):
    scores = (-1)*cross_val_score(ElasticNet(l1_ratio=l1_ratio, tol=0.001, max_iter=1000, random_state=1), X_train, y, cv=5, scoring='neg_mean_squared_error')
    return scores.mean()


In [10]:
results = {}
for i in range(0, 11, 1):
    results[i]=get_score(i*0.1)


C:\Users\szyka\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.849e-01, tolerance: 9.435e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\szyka\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.839e-01, tolerance: 9.143e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of th

In [11]:
results

{0: 0.014453098533445102,
 1: 0.01491560530903736,
 2: 0.018723889285898708,
 3: 0.019127634382155007,
 4: 0.019127634382155007,
 5: 0.019127634382155007,
 6: 0.019127634382155007,
 7: 0.019127634382155007,
 8: 0.019127634382155007,
 9: 0.019127634382155007,
 10: 0.019127634382155007}

We see that for several models there is a warning. Let's try to increase number of iteration to 10000

In [12]:
def get_score_iter(l1_ratio):
    scores = (-1)*cross_val_score(ElasticNet(l1_ratio=l1_ratio, max_iter=10000, random_state=1), X_train, y, cv=5, scoring='neg_mean_squared_error')
    return scores.mean()

"def get_score_iter(l1_ratio):\n    scores = (-1)*cross_val_score(ElasticNet(l1_ratio=l1_ratio, max_iter=10000, random_state=1), X_train, y, cv=5, scoring='neg_mean_squared_error')\n    return scores.mean()"

In [13]:
results_iter = 
for i in range(0, 11, 1):
    results[i]=get_score_iter(i*0.1)

'results_iter = \nfor i in range(0, 11, 1):\n    results[i]=get_score_iter(i*0.1)'

After over 10 minutes we still have some warnings. Maybe best result model (l1_ratio=0) is not giving any problems

In [14]:
model_en = ElasticNet(l1_ratio=0, tol=0.001, max_iter=10000, random_state=1)
model_en.fit(X_train, y)

C:\Users\szyka\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.725e-01, tolerance: 1.229e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


ElasticNet(l1_ratio=0, max_iter=10000, random_state=1, tol=0.001)

We see that we obtain best results when "l1_ratio" is equal to 0. It means that best model for our data set is ridge regression model. In my opinion it is because there is enormously more variables than observations. That is the situation when ridge regression can perform better than Lasso. But in the warning we have information that we can try RidgeCV that is fitted more efficiently. Let's try that model.

In [15]:
from sklearn.linear_model import RidgeCV

In [16]:
model_r = RidgeCV(cv=5)
model_r.fit(X_train, y)

RidgeCV(cv=5)

We see that there is no warnings. Now we will invastigate mse.

In [19]:
prediction=model_r.predict(X_test)

In [21]:
prediction

array([0.97473485, 0.90293189, 0.93184013, 0.95569422, 0.8691048 ,
       0.77426137, 0.9158673 , 0.81362996, 0.88068154, 0.85935401,
       1.05373807, 0.87857979, 0.82332545, 0.63403935, 0.95539089,
       1.03634037, 0.8275145 , 0.94098437, 0.6933974 , 0.77360648,
       0.65057646, 0.88651433, 0.84469309, 0.74017961, 0.88764595,
       0.90325946, 0.88265411, 0.65470248, 0.97456774, 0.68178456,
       0.80793497, 0.94990361, 0.89141129, 0.72763408, 0.89959846,
       0.69222128, 0.91504013, 0.88319335, 0.52354676, 0.65937964,
       0.86887114, 0.7611533 , 0.90206938, 1.0060718 , 0.86870953,
       0.99035289, 0.73197086, 0.66077662, 0.95772352, 0.87399934,
       0.93292172, 0.76721109, 0.96053407, 0.93122423, 0.82426678,
       0.95106247, 0.90435489, 0.97323577, 0.82089764, 0.91927147,
       0.83303758, 0.83484209, 0.9738336 , 0.97913081, 0.93315069,
       0.83975975, 0.96969512, 0.75047593, 0.58279065, 0.90289292,
       0.91631355, 0.38390333, 0.91039448, 0.95269582, 0.94085

Now we will check how Random forest model will perform on our data set

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X_train_rfr = top_500_cols

,ENSG00000129824,ENSG00000184292,ENSG00000163993,ENSG00000122862,ENSG00000171345,ENSG00000204287,ENSG00000118785,ENSG00000230937,ENSG00000133169,ENSG00000167644,...,ENSG00000143333,ENSG00000196735,ENSG00000157214,ENSG00000170476,ENSG00000164294,ENSG00000079112,ENSG00000075643,ENSG00000139289,ENSG00000125850,ENSG00000125726
0,3.182198,4.033507,12.141730,2.870359,12.020272,2.853857,2.976589,2.975880,3.077572,9.829330,...,3.473683,3.088935,3.735464,3.163082,4.089234,9.148311,7.670131,8.681529,6.370924,4.016640
1,2.896301,11.415213,12.823129,2.904678,12.778011,10.726161,2.925108,10.208873,4.193578,9.650949,...,3.393908,3.125673,6.895485,3.991203,5.920513,2.879645,6.931864,6.373925,8.019916,3.965501
2,2.737682,10.666418,3.635745,2.916668,12.108477,3.551805,2.882070,3.084958,3.352948,11.102829,...,3.291920,3.003933,8.325705,3.336496,6.548224,3.004829,3.715563,10.561546,7.056477,3.618266
3,3.512189,10.590466,11.987303,2.938763,11.720451,3.162247,3.016976,7.606048,3.424096,3.062215,...,4.275209,3.057965,2.915628,5.240844,2.677779,3.137427,4.397014,3.442794,4.555335,4.018431
4,2.961343,9.888883,4.718643,2.858609,12.996339,2.814782,2.880148,3.130928,3.237267,2.927342,...,3.394207,2.964157,2.651286,4.214614,6.468536,2.888658,5.218814,3.853191,5.471416,4.065496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,3.786291,11.211629,12.969220,2.956050,12.637708,3.048539,2.969986,3.135039,3.311895,10.415566,...,3.734044,2.937078,8.533649,3.359658,6.021126,9.082506,8.149750,10.410169,5.806246,4.016926
640,4.018057,11.941617,12.676235,8.757200,11.089748,2.941417,3.095832,11.566505,3.333017,9.138738,...,3.656884,3.104766,3.699962,3.629262,6.978750,2.893266,9.378995,7.161075,6.269668,4.122071
641,2.932608,3.003441,9.758864,2.975189,11.360486,3.024615,3.079584,3.034243,3.064740,9.816923,...,5.943107,3.063674,6.729988,3.159595,4.729276,9.484206,10.145304,5.665610,6.153742,4.090919
642,12.119873,11.719849,3.853993,3.151453,12.206070,5.484684,2.936075,7.094722,3.358969,9.965068,...,3.374608,3.074905,6.693722,3.259892,6.199547,3.043489,9.808196,6.602170,7.101028,4.022826


In [ ]:
def get_score_rfr(n_estimators):
    scores = (-1)*cross_val_score(RandomForestRegressor(n_estimators=n_estimators), X_train_rfr, y, cv=5, scoring='neg_mean_squared_error')
    return scores.mean()

In [ ]:
results_rfr = {}
for i in range(10, 110, 10):
    results_rfr[i] = get_score_rfr(i)
results_rfr

{10: 0.013338986333989694,
 20: 0.013066390455454126,
 30: 0.013216604950045025,
 40: 0.012748181938118386,
 50: 0.012598471800869416,
 60: 0.012890534464041318,
 70: 0.012646492288545119,
 80: 0.012738910132888175,
 90: 0.012433046736804234,
 100: 0.012722207644401923}

best model is for n_estimators = 90